In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
df2=pd.read_csv("KVK_Field_sensor1 modified - Soda.csv",parse_dates=['Date Time'])
df2.drop(['Wind direction','Snowfall','Snow depth','Short-wave irradiation'],axis='columns',inplace=True)
temp=df2[['Temperature']]
rel=df2[['Relative Humidity']]
pres=df2[['Pressure']]
wind=df2[['Wind speed']]
rain=df2[['Rainfall']]
df=pd.read_csv("KVK_ORG1.xlsx - feeds.csv",parse_dates=['created_at'])
df['created_at'] = df.created_at.dt.strftime('%Y-%m-%d %H:%M:%S')
df1=df.drop(['Air Temperature (degree Centigrate)','entry_id','Relative Air Humidity (%)','Soil moisture (Resistance-Ohm)'],axis='columns')
merged1=pd.concat([df1,temp,rel,pres,wind,rain],axis='columns')
soil=df[['Soil moisture (Centi Bar)']]
merged2=pd.concat([df2,soil],axis='columns')
merged2

C:\Users\avina\AppData\Local\Temp\ipykernel_1856\13153788.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("KVK_ORG1.xlsx - feeds.csv",parse_dates=['created_at'])


,Date Time,Temperature,Relative Humidity,Pressure,Wind speed,Rainfall,Soil moisture (Centi Bar)
0,01-01-2025 01:00,278.08,63.40,989.24,1.24,0.0,34.02
1,01-01-2025 02:00,278.29,62.04,989.94,1.00,0.0,34.00
2,01-01-2025 03:00,279.06,60.07,990.66,0.88,0.0,34.00
3,01-01-2025 04:00,281.46,56.88,991.56,1.20,0.0,34.00
4,01-01-2025 05:00,284.02,47.38,992.36,1.73,0.0,34.00
...,...,...,...,...,...,...,...
200172,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200173,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200174,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200175,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
merged1.rename(columns = {'created_at':'Date Time'}, inplace = True)
veg=pd.read_csv("Vegetation indices  modified ee-chart.csv",parse_dates=['Vegitation Index'])
veg.drop(0,axis='rows',inplace=True)
veg.rename(columns = {'Vegitation Index':'Date Time'}, inplace = True)
veg.drop(['NDWI1','CAI','EVI','GNDVI','SAVI','MSI','ARI','DWSI','CVI'], axis = 1,inplace=True)
veg.dropna(inplace=True)
merged1['Soil moisture (Centi Bar)'].fillna('0',inplace=True)
merged1['Temperature'].fillna('0',inplace=True)
merged1['Relative Humidity'].fillna('0',inplace=True)
merged1['Pressure'].fillna('0',inplace=True)
merged1['Wind speed'].fillna('0',inplace=True)
merged1['Rainfall'].fillna('0',inplace=True)
merged1.dropna(axis='rows',inplace=True)
merged2.dropna(axis='rows',inplace=True)
veg

C:\Users\avina\AppData\Local\Temp\ipykernel_1856\2815666030.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged1['Soil moisture (Centi Bar)'].fillna('0',inplace=True)
C:\Users\avina\AppData\Local\Temp\ipykernel_1856\2815666030.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged1['Soil moisture (Centi Bar)'].fillna('0',inplace=True)
C:\User

,Date Time,NDVI
1,05-12-2025 12:00,0.335
2,08-12-2025 12:00,0.342
3,10-12-2025 12:00,0.401
4,15-12-2025 12:00,0.356
5,18-12-2025 12:00,0.458
...,...,...
98,13-02-2025 12:00,0.357
99,16-02-2025 12:00,0.357
100,18-02-2025 12:00,0.377
101,21-02-2025 12:00,0.405


In [4]:
merged2=merged2[['Date Time','Soil moisture (Centi Bar)','Temperature','Relative Humidity','Pressure','Wind speed','Rainfall']]
merged1['Date Time'] = pd.to_datetime(merged1['Date Time'], errors='coerce')
merged2['Date Time'] = pd.to_datetime(merged2['Date Time'], format="%d-%m-%Y %H:%M", errors='coerce')
merged2 = merged2.dropna(subset=['Date Time'])
final=pd.merge_asof(merged1,merged2.sort_values(['Date Time','Temperature']),left_on='Date Time',right_on='Date Time')
final['Temperature_x'] = pd.to_numeric(final['Temperature_x'], errors='coerce')
final['Temperature_y'] = pd.to_numeric(final['Temperature_y'], errors='coerce')
final['TEMPERATURE']= final[['Temperature_y','Temperature_x']].median(axis='columns',skipna=True)
final.drop(['Temperature_y','Temperature_x'],axis='columns',inplace=True)
final['Relative Humidity_x'] = pd.to_numeric(final['Relative Humidity_x'], errors='coerce')
final['Relative Humidity_y'] = pd.to_numeric(final['Relative Humidity_y'], errors='coerce')
final['RELATIVE_HUMIDITY']= final[['Relative Humidity_x','Relative Humidity_y']].median(axis='columns',skipna=True)
final.drop(['Relative Humidity_x','Relative Humidity_y'],axis='columns',inplace=True)
final['Soil moisture (Centi Bar)_x'] = pd.to_numeric(final['Soil moisture (Centi Bar)_x'], errors='coerce')
final['Soil moisture (Centi Bar)_y'] = pd.to_numeric(final['Soil moisture (Centi Bar)_y'], errors='coerce')
final['SOIL_MOISTURE']= final[['Soil moisture (Centi Bar)_x','Soil moisture (Centi Bar)_y']].median(axis='columns',skipna=True)
final.drop(['Soil moisture (Centi Bar)_x','Soil moisture (Centi Bar)_y'],axis='columns',inplace=True)
final['Pressure_x'] = pd.to_numeric(final['Pressure_x'], errors='coerce')
final['Pressure_y'] = pd.to_numeric(final['Pressure_y'], errors='coerce')
final['PRESSURE']= final[['Pressure_x','Pressure_y']].median(axis='columns',skipna=True)
final.drop(['Pressure_x','Pressure_y'],axis='columns',inplace=True)
final['Wind speed_x'] = pd.to_numeric(final['Wind speed_x'], errors='coerce')
final['Wind speed_y'] = pd.to_numeric(final['Wind speed_y'], errors='coerce')
final['WIND_SPEED']= final[['Wind speed_x','Wind speed_y']].median(axis='columns',skipna=True)
final.drop(['Wind speed_x','Wind speed_y'],axis='columns',inplace=True)
final['Rainfall_x'] = pd.to_numeric(final['Rainfall_x'], errors='coerce')
final['Rainfall_y'] = pd.to_numeric(final['Rainfall_y'], errors='coerce')
final['RAINFALL']= final[['Rainfall_x','Rainfall_y']].median(axis='columns',skipna=True)
final.drop(['Rainfall_x','Rainfall_y'],axis='columns',inplace=True)
final_ds = pd.merge_asof(merged1, merged2.sort_values(['Date Time', 'Temperature']),left_on='Date Time', right_on='Date Time')
final_ds.dropna(axis='rows',inplace=True)
print(final_ds.head())
final_ds

            Date Time Soil moisture (Centi Bar)_x Temperature_x  \
0 2025-01-25 20:29:44                       34.02        278.08   
1 2025-01-25 20:30:02                        34.0        278.29   
2 2025-01-25 20:30:19                        34.0        279.06   
3 2025-01-25 20:30:37                        34.0        281.46   
4 2025-01-25 20:30:55                        34.0        284.02   

  Relative Humidity_x Pressure_x Wind speed_x Rainfall_x  \
0                63.4     989.24         1.24        0.0   
1               62.04     989.94          1.0        0.0   
2               60.07     990.66         0.88        0.0   
3               56.88     991.56          1.2        0.0   
4               47.38     992.36         1.73        0.0   

   Soil moisture (Centi Bar)_y  Temperature_y  Relative Humidity_y  \
0                        17.92         288.14                 63.7   
1                        17.92         288.14                 63.7   
2                        1

,Date Time,Soil moisture (Centi Bar)_x,Temperature_x,Relative Humidity_x,Pressure_x,Wind speed_x,Rainfall_x,Soil moisture (Centi Bar)_y,Temperature_y,Relative Humidity_y,Pressure_y,Wind speed_y,Rainfall_y
0,2025-01-25 20:29:44,34.02,278.08,63.4,989.24,1.24,0.0,17.92,288.14,63.70,983.96,2.45,0.019685
1,2025-01-25 20:30:02,34.0,278.29,62.04,989.94,1.0,0.0,17.92,288.14,63.70,983.96,2.45,0.019685
2,2025-01-25 20:30:19,34.0,279.06,60.07,990.66,0.88,0.0,17.92,288.14,63.70,983.96,2.45,0.019685
3,2025-01-25 20:30:37,34.0,281.46,56.88,991.56,1.2,0.0,17.92,288.14,63.70,983.96,2.45,0.019685
4,2025-01-25 20:30:55,34.0,284.02,47.38,992.36,1.73,0.0,17.92,288.14,63.70,983.96,2.45,0.019685
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200172,2025-03-16 12:40:28,0,0,0,0,0,0,17.80,284.60,66.11,984.00,1.21,0.000000
200173,2025-03-16 12:40:46,0,0,0,0,0,0,17.80,284.60,66.11,984.00,1.21,0.000000
200174,2025-03-16 12:41:04,0,0,0,0,0,0,17.80,284.60,66.11,984.00,1.21,0.000000
200175,2025-03-16 12:41:21,0,0,0,0,0,0,17.80,284.60,66.11,984.00,1.21,0.000000


In [5]:
final_ds.to_csv("C:/Users/avina/OneDrive/Desktop/python/final_ds.csv", index=False)
